In [71]:
import pickle
import numpy as np
from keras.models import Sequential
import keras.layers
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import tensorflow as tf

In [23]:
np.random.seed(0)
train, valid, test = pickle.load(open('dataset.pickle', 'rb'))
train = train.fillna(0)

In [69]:
x_train = train[['raw_acc','proc_gyro','raw_magnet','watch_acceleration','watch_heading', 'location', 'location_quick_features', 'audio_naive', 'audio_properties', 'discrete', 'lf_measurements']]
y_train = train.iloc[:,-8:]

In [72]:
scaler = MinMaxScaler() #scale features between 0 and 1
x_train = scaler.fit_transform(x_train)
x_train = np.asarray(x_train).reshape(x_train.shape[0], 1, x_train.shape[1])

In [73]:
model = Sequential()
model.add(LSTM(500, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dense(2000, activation='relu'))
model.add(Dropout(0.2, noise_shape=None, seed=None))
model.add(BatchNormalization())
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [74]:
history = model.fit([x_train], batch_size=20, y=y_train, verbose=1, validation_split=0.25, 
          shuffle=True, epochs=5)

Train on 227768 samples, validate on 75923 samples
Epoch 1/5
227768/227768 [==============================] - 150s 660us/step - loss: 1.0040 - acc: 0.6360 - val_loss: 1.4033 - val_acc: 0.5736
Epoch 2/5
227768/227768 [==============================] - 153s 673us/step - loss: 0.7942 - acc: 0.7127 - val_loss: 1.3533 - val_acc: 0.5477
Epoch 3/5
227768/227768 [==============================] - 167s 735us/step - loss: 0.7017 - acc: 0.7475 - val_loss: 1.5726 - val_acc: 0.4997
Epoch 4/5
227768/227768 [==============================] - 173s 759us/step - loss: 0.6472 - acc: 0.7690 - val_loss: 1.5980 - val_acc: 0.5115
Epoch 5/5
227768/227768 [==============================] - 168s 737us/step - loss: 0.6124 - acc: 0.7814 - val_loss: 1.5745 - val_acc: 0.5204


In [64]:
print(x_train.shape, y_train.shape)


(303691, 1, 225) (303691, 8)


In [100]:
import pandas as pd
pd.isna(train).sum(axis = 1).sum()

0